# Basic OLS


## Loading Packages

In [1]:
using Dates, DelimitedFiles, Statistics, LinearAlgebra, Distributions

include("jlFiles/printmat.jl")
include("jlFiles/NWFn.jl")

NWFn

## Loading Data

In [2]:
x = readdlm("Data/FFmFactorsPs.csv",',',skipstart=1)

                #yearmonth, market, small minus big, high minus low
(ym,Rme,RSMB,RHML) = (x[:,1],x[:,2]/100,x[:,3]/100,x[:,4]/100) 
x = nothing                   

println(size(Rme))

(388,)


## Point Estimates

Consider the linear regression

$
y_{t}=\beta^{\prime}x_{t}+u_{t},
$

where $y_{t}$ is a scalar and $x_{t}$ is $k\times1$. The OLS estimate is

$
\hat{\beta} = S_{xx}^{-1}S_{xy}, \: \text{ where } \: 
S_{xx}      = \frac{1}{T}\sum\nolimits_{t=1}^{T}x_{t}x_{t}^{\prime}
\: \text{ and } \:
S_{xy}      = \frac{1}{T}\sum\nolimits_{t=1}^{T}x_{t}y_{t}.
$

(The $1/T$ terms clearly cancel.)

Instead of these sums (loops over $t$), matrix multiplication can be used to
speed up the calculations. Create matrices $X_{T\times k}$ and $Y_{T\times1}$
by letting $x_{t}^{\prime}$ and $y_{t}$ be the $t^{th}$ rows

$
X_{T\times k}=\left[
\begin{array}[c]{l}
x_{1}^{\prime}\\
\vdots\\
x_{T}^{\prime}
\end{array}
\right] \ \text{ and } \ Y_{T\times1}=\left[
\begin{array}[c]{l}
y_{1}\\
\vdots\\
y_{T}
\end{array}
\right].
$

We can then calculate the same matrices as

$
S_{xx}       =X^{\prime}X/T \ \text{ and } \: S_{xy}=X^{\prime}Y/T \: \text{, so } \: 
\hat{\beta}  =(X^{\prime}X)^{-1}X^{\prime}Y.
$

However, instead of inverting $S_{xx}$, we typically get much better numerical
precision by solving the system of $T$ equations

$
X_{T\times k}b_{k\times1}=Y_{T\times1}
$

for the vector $b$ that minimizes the sum of squared errors. This
is easily done by using the command
```
b = X\Y
```

In [3]:
println("Three different ways to calculate OLS estimates")

Y = Rme
T = size(Y,1)
X = [ones(T) RSMB RHML]

K    = size(X,2)
S_xx = zeros(K,K)
S_xy = zeros(K,1)
for t = 1:T
    local x_t, y_t
    global S_xx, S_xy
    x_t = X[t,:]               #a vector
    y_t = Y[t:t,:]             
    S_xx = S_xx + x_t*x_t'/T   #KxK
    S_xy = S_xy + x_t*y_t/T    #Kx1
end
b1 = inv(S_xx)*S_xy          #OLS coeffs, version 1

b2 = inv(X'X)*X'Y            #OLS coeffs, version 2

b3 = X\Y                     #OLS coeffs, version 3

println("\n      b1       b2        b3")
printmat([b1 b2 b3])

Three different ways to calculate OLS estimates

      b1       b2        b3
     0.007     0.007     0.007
     0.217     0.217     0.217
    -0.429    -0.429    -0.429



## Distribution of OLS Estimates

The distribution of the estimates is (typically)

$
(\hat{\beta}-\beta_{0})\overset{d}{\rightarrow}N(0,V)
\: \text{ where } \: V=S_{xx}^{-1} S S_{xx}^{-1}/T
$

where $S$ is the covariance matrix of $\sqrt{T}\bar{g}$, where $\bar{g}$
is the sample average of

$
g_{t}=x_t u_t,
$

and $S_{xx}$ is defined as 

$
S_{xx}=-\sum_{t=1}^{T}x_{t}x_{t}^{\prime}/T.
$

When $x_t$ and $u_t$ are independent (Gauss-Markov assumptions...), then $S$ can be simplified as $S=S_{xx}\sigma^2$, where $\sigma^2$ is the variance of $u_t$. Clearly, this means that $V$ can be written $V=S_{xx}^{-1}\sigma^2/T$.

In [4]:
b   = X\Y
u   = Y - X*b              #residuals
g   = X.*u                 #TxK
Sxx = X'X/T

V_iid = inv(Sxx)*var(u)/T

S     = NWFn(g,1)               #Newey-West covariance matrix
V_NW  = inv(Sxx)'S*inv(Sxx)/T   #Cov(b)

println("\n     b       std_iid  std_NW")
printmat([b3 sqrt.(diag(V_iid))   sqrt.(diag(V_NW))])


     b       std_iid  std_NW
     0.007     0.002     0.002
     0.217     0.073     0.124
    -0.429     0.074     0.108



## A Function for OLS

In [5]:
"""
    OlsFn(y,x,m=1)

LS of y on x; for one dependent variable

# Usage
(b,res,yhat,V,R2a) = OlsFn(y,x,m)

# Input
- `y::Array`:     Tx1, the dependent variable
- `x::Array`:     Txk matrix of regressors (including deterministic ones)
- `m::Int`:       scalar, bandwidth in Newey-West  

# Output
- `b::Array`:     kx1, regression coefficients
- `u::Array`:     Tx1, residuals y - yhat
- `yhat::Array`:  Tx1, fitted values x*b
- `V::Array`:     kxk matrix, covariance matrix of b
- `R2a::Number`:  scalar, R2 value

"""
function OlsFn(y,x,m=0)
    T    = size(y,1)
    b    = x\y
    yhat = x*b
    u    = y - yhat
    g    = x.*u   
    S0   = NWFn(g,m)            #Newey-West covariance matrix
    Sxx  = x'x/T
    V    = inv(Sxx)'S0*inv(Sxx)/T  
    R2a  = 1 - var(u)/var(y)
    return b,u,yhat,V,R2a
end

OlsFn

In [6]:
(b4,_,_,V,R2a) = OlsFn(Y,X,1)
println("\n with NW standard errors")
printmat([b4 sqrt.(diag(V))])


 with NW standard errors
     0.007     0.002
     0.217     0.124
    -0.429     0.108



## Testing a Hypothesis

Since the estimator $\hat{\beta}_{_{k\times1}}$ satisfies

$
\hat{\beta}-\beta_{0} \sim N(0,V_{k\times k})  ,
$

we can easily apply various tests. To test a joint linear hypothesis of the
form

$
\gamma_{q\times1}=R\beta-a,
$

use the test

$
(R\beta-a)^{\prime}\Lambda ^{-1}(R\beta
-a)\overset{d}{\rightarrow}\chi_{q}^{2} \: \text{, where } \: \Lambda=RVR^{\prime}.
$

In [7]:
R = [0 1 0;               #testing if b(2)=0 and b(3)=0
     0 0 1]
a = [0;0]
Γ = R*V*R'
test_stat = (R*b-a)'inv(Γ)*(R*b-a)
println("\ntest-statictic and 10% critical value of chi-square(2)")
printmat([test_stat quantile(Chisq(2),0.9)])


test-statictic and 10% critical value of chi-square(2)
    26.059     4.605

